In [1]:
import gensim

In [2]:
text_corpus = [
    "Human machine interface for lab abc computer applications",
    "A survey of user opinion of computer system response time",
    "The EPS user interface management system",
    "System and human system engineering testing of EPS",
    "Relation of user perceived response time to error measurement",
    "The generation of random binary unordered trees",
    "The intersection graph of paths in trees",
    "Graph minors IV Widths of trees and well quasi ordering",
    "Graph minors A survey",
]

In [8]:
# tokenizer
class WhitespaceTokenizer():
    def tokenize(self, input:str) -> list[str]:
        if isinstance(input, str):
            result = input.split(" ")
        return result

# Text Cleaner
class TextCleaner:
    def __init__(self):
        # Create a set of frequent words
        self.stopwords = set('for a of the and to in'.split(' '))
    def clean_text(self, words:list[str]) -> list[str]:
        # Lowercase each document, split it by white space and filter out stopwords
        words = [word.lower() for word in words if word.lower() not in self.stopwords]
        return words

# filter by frequency
class FilterByFrequency:
    def __init__(self):
        # Count word frequencies
        from collections import defaultdict
        self.frequency_dict = defaultdict(int)
    def make_filter(self, docs:list[list[str]]):
        for text in docs:
            for token in text:
                self.frequency_dict[token] += 1
    def filter(self, words:list[str], threshold:int=1):
        # Only keep words that appear more than once
        filtered_words = [token for token in words if self.frequency_dict[token] > threshold]
        return filtered_words

# bow
class BagOfWords:
    def __init__(self):
        self.dictionary:dict[str,int]|None=None
    def create_dictionary(self, input:list[list[str]]):
        from gensim import corpora
        self.dictionary = corpora.Dictionary(input)
    def represent_bow(self, input:list[list[str]]):
        bow_corpus = [self.dictionary.doc2bow(text) for text in input]
        return bow_corpus

In [16]:
# (1) 토크나이징 : 공백을 기준으로
tokenizer = WhitespaceTokenizer()
tokenized_docs = [tokenizer.tokenize(doc) for doc in text_corpus]
# (2) 텍스트 클리닝 - lower + stopwords
text_cleaner = TextCleaner()
cleaned_docs = [text_cleaner.clean_text(words) for words in tokenized_docs]
# (3) 빈도 기반 필터링 : 1회 발생 단어는 제외
filter = FilterByFrequency()
filter.make_filter(cleaned_docs)
processed_corpus = [filter.filter(doc, 1) for doc in cleaned_docs]
# (4) BoW 생성
bow_model = BagOfWords()
bow_model.create_dictionary(processed_corpus)
bow = bow_model.represent_bow(processed_corpus)

print("===== corpus(words) =====")
print(processed_corpus)
print("===== BoW =====")
print(bow)

===== corpus(words) =====
[['human', 'interface', 'computer'], ['survey', 'user', 'computer', 'system', 'response', 'time'], ['eps', 'user', 'interface', 'system'], ['system', 'human', 'system', 'eps'], ['user', 'response', 'time'], ['trees'], ['graph', 'trees'], ['graph', 'minors', 'trees'], ['graph', 'minors', 'survey']]
===== BoW =====
[[(0, 1), (1, 1), (2, 1)], [(0, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1)], [(2, 1), (5, 1), (7, 1), (8, 1)], [(1, 1), (5, 2), (8, 1)], [(3, 1), (6, 1), (7, 1)], [(9, 1)], [(9, 1), (10, 1)], [(9, 1), (10, 1), (11, 1)], [(4, 1), (10, 1), (11, 1)]]


In [12]:
# 새로운 문장 BoW
new_sentence = "Human computer interaction"
cleaned_words = filter.filter(text_cleaner.clean_text(tokenizer.tokenize(new_sentence)))
new_vec = bow_model.dictionary.doc2bow(cleaned_words)
print("===== BoW =====")
print(new_vec)
print("===== BoW Token Dictionary =====")
print(bow_model.dictionary.token2id)

===== BoW =====
[(0, 1), (1, 1)]
===== BoW Token Dictionary =====
{'computer': 0, 'human': 1, 'interface': 2, 'response': 3, 'survey': 4, 'system': 5, 'time': 6, 'user': 7, 'eps': 8, 'trees': 9, 'graph': 10, 'minors': 11}


In [14]:
processed_corpus

[['human', 'interface', 'computer'],
 ['survey', 'user', 'computer', 'system', 'response', 'time'],
 ['eps', 'user', 'interface', 'system'],
 ['system', 'human', 'system', 'eps'],
 ['user', 'response', 'time'],
 ['trees'],
 ['graph', 'trees'],
 ['graph', 'minors', 'trees'],
 ['graph', 'minors', 'survey']]